In [ ]:
!pip install tensorflow-gpu==2.10.0
!pip install cudnn==8.4.1
!pip install cudatoolkit==11.8.0
!pip install pillow
!pip install scikit-learn
!pip install openpyxl
!pip install opencv
!pip install pandas
!pip install matplotlib

# 新段落

In [ ]:
import tensorflow as tf
tf.__version__        # 此命令为获取安装的tensorflow版本
print(tf.__version__) # 输出版本
tf.__path__			   #查看tensorflow安装路径
print(tf.__path__)

2.10.0
['/usr/local/lib/python3.10/dist-packages/keras/api/_v2', '/usr/local/lib/python3.10/dist-packages/tensorflow_estimator/python/estimator/api/_v2', '/usr/local/lib/python3.10/dist-packages/tensorboard/summary/_tf', '/usr/local/lib/python3.10/dist-packages/tensorflow', '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# 切换到目标目录
os.chdir('/content/drive/MyDrive/rdkit')
# 打印当前工作目录
print("当前工作目录：", os.getcwd())

当前工作目录： /content/drive/MyDrive/rdkit


## All parameters and file paths are set in the next cell.

In [ ]:
# Focal Loss Parameters
gamma_values = [1]   # Typically between 1.0 and 5.0, increase the focus on samples that are more difficult to classify.
pos_weight_values = [1]  # Values less than 1.0 emphasize positive samples in imbalanced datasets where positive samples are rare.
# BiLSTM Layer Parameters
lstm_units1_values = [32]
num_lstm_layers_values = [1]
# Dense Layer Parameters
dense_units1_values = [32]
dense_layers_values = [3]
# Dropout Layer Parameters
dropout_rate1_values = [0.3]
dropout_layers_values = [1]

# Set the number of random iterations
num_iterations = 10
# training set and independent test set path setting
train_file = 'Dataset/24_IL-6/IL6_train.xlsx'
val_file = 'Dataset/24_IL-6/IL6_test.xlsx'

## The following cell imports the necessary modules and defines custom function for training the model. The code does not require any modification.

In [ ]:
import cv2
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, roc_curve, roc_auc_score
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold
from preprocess_data_test import preprocess_seq, get_max_length, preprocess_sequence
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, balanced_accuracy_score, roc_curve, \
    matthews_corrcoef, precision_score, recall_score, f1_score
import math
from statistics import stdev
import random
import logging
from focal_loss import BinaryFocalLoss

def map_fn(sequence, label):
    processed_sequence = tf.py_function(preprocess_sequence, [sequence], tf.float32)
    return processed_sequence, label
def load_and_preprocess_data(sequences, labels):
    sequences = tf.constant(sequences, dtype=tf.string)
    labels = tf.constant(labels, dtype=tf.float32)

    # 创建 tf.data.Dataset，并使用 map 应用处理函数
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    dataset = dataset.map(lambda sequence, label: map_fn(sequence, label), num_parallel_calls=tf.data.AUTOTUNE)
    #dataset = dataset.shuffle(buffer_size=len(sequences)).cache()
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(16)
    return dataset

# 自定义回调函数用于打印指标
class PrintMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}:")
        print(f"  loss: {logs['loss']:.4f}")
        print(f"  accuracy: {logs['accuracy']:.4f}")
        print(f"  val_loss: {logs['val_loss']:.4f}")
        print(f"  val_accuracy: {logs['val_accuracy']:.4f}")
        print(f"  learning_rate: {tf.keras.backend.get_value(model.optimizer.lr):.6f}")

class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.context_vector = self.add_weight(shape=(input_shape[-1], 1), initializer='glorot_uniform',
                                              trainable=True)

    def call(self, inputs):
        # Compute attention scores
        attention_scores = tf.matmul(inputs, self.context_vector)
        attention_scores = tf.squeeze(attention_scores, axis=-1)
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)

        # Apply attention weights to inputs
        weighted_inputs = tf.multiply(inputs, tf.expand_dims(attention_weights, axis=-1))
        context_vector = tf.reduce_sum(weighted_inputs, axis=1)

        return context_vector, attention_weights

# 创建一个回调函数，用于在每个epoch结束时打印相关信息
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(
            f"Epoch {epoch + 1} - Train Loss: {logs['loss']:.4f}, Train Accuracy: {logs['accuracy']:.4f}, Test Loss: {logs['val_loss']:.4f}, Test Accuracy: {logs['val_accuracy']:.4f}")

# 定义学习率回调函数，用于在每个epoch结束时打印学习率
class LearningRateCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr.numpy()
        print(f"Epoch {epoch + 1} - Learning Rate: {lr:.6f}")
# 在模型定义后，进行编译之前，定义Focal  Loss类
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2, alpha=0.5):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha

    def call(self, y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
        modulating_factor = tf.pow(1.0 - p_t, self.gamma)
        focal_loss = -alpha_factor * modulating_factor * tf.math.log(p_t)
        return focal_loss

def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.6
    epochs_drop = 3.0
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate
    # Define the training process
def train_model(train_dataset, test_dataset):
    strategy = tf.distribute.MirroredStrategy(devices=['GPU:0', 'GPU:1'])
    with strategy.scope():
        # Define ModelCheckpoint callback
        checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True,
                                                        mode='max', verbose=0)
        # 定义 EarlyStopping 回调函数
        early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True, verbose=1)

        learning_rate = 0.0001
        model = create_model()
        model.optimizer.lr.assign(learning_rate)
        print_learning_rate = tf.keras.callbacks.LambdaCallback(
            on_epoch_begin=lambda epoch, logs: print(
                f"Learning Rate: {tf.keras.backend.get_value(model.optimizer.lr):.6f}"))
        callbacks = [reduce_lr, early_stopping, print_learning_rate,checkpoint]
        model.fit(train_dataset, epochs=400, batch_size=16, validation_data=test_dataset,
                  callbacks=callbacks, verbose=2)
    return model
def create_model():
    learning_rate = 0.0001
    momentum = 0.5
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay)
    # 输入序列的长度
    sequence_length = max_length
    input_shape = (sequence_length, 32, 32, 3)
    inputs = tf.keras.Input(shape=input_shape)
    # model = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(64, (1, 1), activation='relu', padding='same'))(
    initializer = tf.keras.initializers.HeNormal(seed=123456)
    model = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))(
        inputs)
    for _ in range(2):
        model = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))(
            inputs)
        model = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)))(model)
    for _ in range(1):
        model = tf.keras.layers.BatchNormalization()(model)  # Adding a normalization layer
    model = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same'))(model)
    for _ in range(2):
        model = tf.keras.layers.TimeDistributed(
            tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same'))(model)
        model = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)))(model)
    for _ in range(1):
        model = tf.keras.layers.BatchNormalization()(model)  # Adding a normalization layer

    model = tf.keras.layers.Reshape((sequence_length, -1))(inputs)
    lstm_units = lstm_units1
    # 根据num_lstm_layers创建指定层数的LSTM层
    for _ in range(num_lstm_layers):
        model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_units1, return_sequences=True))(model)
    # 添加 Temporal Attention 机制
    model = tf.keras.layers.Reshape((-1, lstm_units * 2))(model)
    permute1 = tf.keras.layers.Permute((2, 1))(model)
    attention_probs = tf.keras.layers.Dense(units=1, activation='softmax')(permute1)
    permute2 = tf.keras.layers.Permute((2, 1))(attention_probs)
    model = tf.keras.layers.Multiply()([model, permute2])
    model = tf.keras.layers.Flatten()(model)
    # 添加全连接层和dropout层
    for _ in range(dense_layers):
        model = tf.keras.layers.Dense(units=dense_units1)(model)
    for _ in range(dropout_layers):
        model = tf.keras.layers.Dropout(rate=dropout_rate1)(model)
    num_classes = 1
    outputs = tf.keras.layers.Dense(units=num_classes, activation='sigmoid')(model)
    # 创建模型
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    sgd = tf.keras.optimizers.SGD(lr=0.1, momentum=momentum, decay=0.0, nesterov=False)
    rmsprop = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    # 编译模型
    model.compile(optimizer=adam, loss=BinaryFocalLoss(gamma=gamma,pos_weight=pos_weight), metrics=['accuracy'])  # rmsprop adam
    # 打印模型结构
    model.summary()
    return model
def preprocess_sequence(sequence):
    # 读取所有照片并将它们存储在一个字典中
    images = {}
    folder_path = "residues32/IA"
    file_names = os.listdir(folder_path)

    # 加载和预处理图像
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        image = cv2.imread(file_path)
        image = cv2.resize(image, (32, 32))  # 调整图像大小为 32x32
        image = tf.cast(image, tf.float32) / 255.0  # 转换为 float32 并标准化到 [0, 1] 范围内
        # # 应用 Z-分数标准化（减去均值并除以标准差）
        # mean = tf.math.reduce_mean(image)
        # std = tf.math.reduce_std(image)
        # image = (image - mean) / std

        # 将NaN替换为0
        image = tf.where(tf.math.is_nan(image), tf.zeros_like(image), image)
        images[file_path[14:-4]] = image
    def map_seq(input_str):
        # print(input_str)
        # # 查看图像的形状以调试
        char_images = []  # 创建一个空列表
        prev_index = None
        # 遍历输入字符串中的每个字符
        for index, char in enumerate(input_str):
            # 如果当前字符是 "x"，记住它的索引并退出循环
            if char == 'x':
                prev_index = index-1
                break
        # 如果没有找到 "x"，则记住最后一个字符的索引
        if prev_index is None and len(input_str) > 0:
            prev_index = len(input_str) - 1

        # 遍历输入字符串中的每个字符
        for n in range(len(input_str)):
            # for char in input_str[n:n+1]:
            if n == prev_index:
                char = input_str[n]
                image_key = char + '_C'
                #print(image_key)
                char_tensor = tf.convert_to_tensor(images.get(image_key))
                char_images.append(char_tensor)
            elif n == 0:
                char = input_str[n]
                image_key = char + '_N'
                char_tensor = tf.convert_to_tensor(images.get(image_key))
                char_images.append(char_tensor)
            # 检查字符是否在images字典中
            elif n != prev_index:
                char = input_str[n]
                #print(char)
                # 如果在images字典中，将对应的图像转换为Tensor并添加到列表中
                char_tensor = tf.convert_to_tensor(images.get(char))
                char_images.append(char_tensor)
        char_images = np.array(char_images)

        seq_frames = tf.stack(char_images, axis=0)
        return seq_frames

    input_seq = sequence.numpy().decode("utf-8")
    #print("input sequence"+input_seq)
    processed_data = []
    # for seq in input_seq:
    #     print(seq)
    seq_frames = map_seq(input_seq)
    processed_data.append(seq_frames)
    processed_data = tf.convert_to_tensor(processed_data)
    #processed_data = tf.squeeze(processed_data, axis=1)
    return processed_data

def preprocess_seq(filename, max_length):
    data = pd.read_excel(filename, engine='openpyxl', keep_default_na=False, na_values=[''])
    sequences = data['sequence'].tolist()
    labels = data['label'].tolist()
    # print(len(sequences))
    # print(len(labels))
    processed_data = []

    for seq, label in zip(sequences, labels):
        # 移除行尾空格并填充到指定长度
        seq = seq.strip().ljust(max_length, 'x')
        processed_data.append((seq, label))  # 将数据和标签打包成一个元组并添加到列表中
    #print(processed_data)
    return processed_data

def get_max_length(filename1,filename2,max_length):

    def count_max_length(data):
        sequences = data['sequence'].tolist()
        labels = data['label'].tolist()
        max_length = 0
        positive_sequences = []
        negative_sequences = []
        for seq, label in zip(sequences, labels):
            if label == 1:
                positive_sequences.append(seq)
            else:
                negative_sequences.append(seq)
            max_length = max(max_length, len(seq))
        return max_length
    # 从文件中读取每一行并将其与相应的照片相关联
    data1 = pd.read_excel(filename1, engine='openpyxl', keep_default_na=False, na_values=[''])  # 指定engine为'openpyxl'或'xlrd'
    data2 = pd.read_excel(filename2, engine='openpyxl', keep_default_na=False, na_values=[''])
    max_length1 = count_max_length(data1)
    max_length2 = count_max_length(data2)
    if max_length1>max_length:
        max_length =max_length1
    if max_length2>max_length1:
        max_length=max_length2
    print("数据集中字符最大长度:", max_length)
    return max_length

## The following cell contains the main training code. After training, the results and hyperparameter settings are saved in the "result.xlsx" file.

In [ ]:
# 设置日志级别
logging.basicConfig(level=logging.INFO)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# 设置日志级别为只输出错误信息和警告
tf.get_logger().setLevel('ERROR')
best_accuracy = 0.0
best_combination = {}
# 生成不重复的随机索引
random_indices = random.sample(range(num_iterations), num_iterations)

# 创建一个空的DataFrame来存储结果
results_df = pd.DataFrame(columns=[
    'Iteration', 'gamma', 'pos_weight', 'lstm_units1', 'dense_units1',
    'num_lstm_layers', 'dense_layers', 'dropout_rate1', 'dropout_layers', 'TP', 'FP', 'FN', 'TN', 'ACC',
    'BACC', 'Sn', 'Sp', 'MCC', 'AUC', "AUC_prime", 'Preci sion', 'Recall', 'F1_score'
])

for i in random_indices:
    gamma = gamma_values[i % len(gamma_values)]
    pos_weight = pos_weight_values[i % len(pos_weight_values)]
    lstm_units1 = lstm_units1_values[i % len(lstm_units1_values)]
    dense_units1 = dense_units1_values[i % len(dense_units1_values)]
    num_lstm_layers = num_lstm_layers_values[i % len(num_lstm_layers_values)]
    dense_layers = dense_layers_values[i % len(dense_layers_values)]
    dropout_rate1 = dropout_rate1_values[i % len(dropout_rate1_values)]
    dropout_layers = dropout_layers_values[i % len(dropout_layers_values)]

    max_length = 0
    max_length = get_max_length(train_file, val_file, max_length)
    print(max_length)
    data_train =preprocess_seq(train_file,max_length)
    data_test =preprocess_seq(val_file,max_length)

    X = np.array([sequence for sequence, label in data_train])
    y = np.array([label for sequence, label in data_train])
    X_val = np.array([sequence for sequence, label in data_test])
    y_val = np.array([label for sequence, label in data_test])

    val_dataset =load_and_preprocess_data(X_val, y_val)
    # 定义五折交叉验证
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # 定义模型保存路径
    model_path = 'best_model'+ str(i+1)+'.h5'
    # best_model = model = tf.keras.models.load_model('best_model.h5')
    best_model = None
    best_accuracy = 0
    # 定义空列表用于存储每个k折训练的指标
    accuracy_list = []
    auc_list = []
    bacc_list = []
    sensitivity_list = []
    specificity_list = []
    mcc_list = []
    precision_list = []
    recall_list = []
    f1_list = []

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1,
                                                     min_delta=1e-4, mode='min')
    # 设置回调函数
    custom_callback = CustomCallback()
    lr_callback = LearningRateCallback()
    callbacks = [custom_callback, lr_callback]

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 确保训练集和测试集的样本数量是批量大小的整数倍
        train_samples = len(X_train) - (len(X_train) % 16)
        test_samples = len(X_test) - (len(X_test) % 16)
        X_train = X_train[:train_samples]
        y_train = y_train[:train_samples]
        X_test = X_test[:test_samples]
        y_test = y_test[:test_samples]
        assert X_test.shape[0] == y_test.shape[0]

        train_dataset = load_and_preprocess_data(X_train, y_train)
        test_dataset =load_and_preprocess_data(X_test,y_test)

        model = train_model(train_dataset,test_dataset)
        # 评估模型在验证集上的性能
        val_loss, val_acc = model.evaluate(val_dataset)
        print("最佳模型的ACC"+str(val_acc))
        print("最佳模型的loss" + str(val_loss))
        # 保存最佳模型
        if val_acc > best_accuracy:
            best_model = model
            best_accuracy = val_acc

        # 初始化空的y_true和y_pred_binary列表
        y_true = []
        y_pred_binary = []

        # 遍历val_dataset并提取标签
        for batch_inputs, batch_labels in val_dataset:
            # 预测每个批次的输出
            batch_inputs = np.squeeze(batch_inputs, axis=1)
            batch_predictions = model.predict(batch_inputs)
            batch_predictions_binary = (batch_predictions > 0.5).astype("int32")

            # 将每个批次的标签和预测值添加到列表中
            y_true.extend(batch_labels.numpy())
            y_pred_binary.extend(batch_predictions_binary)
            # print(y_true)

        # 转换为NumPy数组
        y_true = np.array(y_true)
        y_pred_binary = np.array(y_pred_binary)

        # 计算各项指标
        precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred_binary, zero_division=1)
        fpr, tpr, thresholds = roc_curve(y_true, y_pred_binary)
        cm = confusion_matrix(y_true, y_pred_binary)
        TN, FP, FN, TP = cm.ravel()
        accuracy = accuracy_score(y_true, y_pred_binary)
        auc = roc_auc_score(y_true, y_pred_binary)
        auc_prime = np.trapz(tpr, fpr)
        sensitivity = recall_score(y_true, y_pred_binary)
        specificity = recall_score(y_true, y_pred_binary, pos_label=0)
        mcc = matthews_corrcoef(y_true, y_pred_binary)
        bacc = balanced_accuracy_score(y_true, y_pred_binary)
        precision = precision_score(y_true, y_pred_binary)
        recall = sensitivity
        f1 = f1_score(y_true, y_pred_binary)

        # 将每个k折训练的指标添加到列表中
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        bacc_list.append(bacc)
        sensitivity_list.append(sensitivity)
        specificity_list.append(specificity)
        mcc_list.append(mcc)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    # 计算各项指标的标准差
    accuracy_std = stdev(accuracy_list)
    auc_std = stdev(auc_list)
    bacc_std = stdev(bacc_list)
    sensitivity_std = stdev(sensitivity_list)
    specificity_std = stdev(specificity_list)
    mcc_std = stdev(mcc_list)
    precision_std = stdev(precision_list)
    recall_std = stdev(recall_list)
    f1_std = stdev(f1_list)

    print('Best model accuracy:', best_accuracy)
    # 打印各项指标及其标准差
    print('Accuracy:', accuracy_list)
    print('Accuracy (std):', accuracy_std)
    print('AUC:', auc_list)
    print('AUC (std):', auc_std)
    print('Balanced Accuracy:', bacc_list)
    print('Balanced Accuracy (std):', bacc_std)
    print('Sensitivity:', sensitivity_list)
    print('Sensitivity (std):', sensitivity_std)
    print('Specificity:', specificity_list)
    print('Specificity (std):', specificity_std)
    print('MCC:', mcc_list)
    print('MCC (std):', mcc_std)
    print('Precision:', precision_list)
    print('Precision (std):', precision_std)
    print('Recall:', recall_list)
    print('Recall (std):', recall_std)
    print('F1:', f1_list)
    print('F1 (std):', f1_std)
    best_model.save('best_model'+str(i + 1)+'.h5')
    # 在这里执行您的代码，使用当前的参数组合
    print(f"Iteration {i + 1}/{num_iterations}: "
          f"gamma={gamma}, pos_weight={pos_weight}, "
          f"lstm_units1={lstm_units1}, dense_units1={dense_units1}, "
          f"num_lstm_layers={num_lstm_layers}, dense_layers={dense_layers}, "
          f"dropout_rate1={dropout_rate1},"
          f"dropout_layers={dropout_layers}")
    # 打印结果
    print("True Positive (TP):", TP)
    print("False Positive (FP):", FP)
    print("False Negative (FN):", FN)
    print("True Negative (TN):", TN)
    print("Accuracy (ACC):", accuracy)
    print("Balanced Accuracy (BACC):", bacc)
    print("Sensitivity (Sn):", sensitivity)
    print("Specificity (Sp):", specificity)
    print("Matthews Correlation Coefficient (MCC):", mcc)
    print("Area Under the Curve (AUC):", auc)
    print("AUC' (AUC prime):", auc_prime)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    val_accuracy = accuracy
    # 收集val_accuracy
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_combination = {
            'gamma': gamma,
            'pos_weight': pos_weight,
            'lstm_units': lstm_units1,
            'dense_units': dense_units1,
            'num_lstm_layers': num_lstm_layers,
            'dense_layers': dense_layers,
            'dropout_rate': dropout_rate1,
        }
    #print(pd.__version__)
    #print(type(results_df))# 将参数和评估指标添加到DataFrame中
    results_df = pd.concat([results_df, pd.DataFrame({
        'Iteration': [i + 1],
        'gamma': [gamma],
        'pos_weight': [pos_weight],
        'lstm_units1': [lstm_units1],
        'dense_units1': [dense_units1],
        'num_lstm_layers': [num_lstm_layers],
        'dense_layers': [dense_layers],
        'dropout_rate1': [dropout_rate1],
        'dropout_layers': [dropout_layers],
        'TP': [TP],
        'FP': [FP],
        'FN': [FN],
        'TN': [TN],
        'ACC': [accuracy],
        'BACC': [bacc],
        'Sn': [sensitivity],
        'Sp': [specificity],
        'MCC': [mcc],
        'AUC': [auc],
        'AUC_prime': [auc_prime],
        'Precision': [precision],
        'Recall': [recall],
        'F1_score': [f1]
    })], ignore_index=True)

    # 将DataFrame写入Excel文件
    output_path = 'results.xlsx'
    results_df.to_excel(output_path, index=False)
    print(f"Results saved to {output_path}")

# 打印最佳结果的组合和最佳准确率
print(f"Best Combination: {best_combination}")
print(f"Best Accuracy: {best_accuracy}")

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


数据集中字符最大长度: 180
